In [1]:
# TO DO
# add to DAPI
# front end UI
# fix df inventory information
# backcheck cut in calculations

In [2]:
import sys
sys.path.append("../decklizer")
from engine import *

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy

from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [28]:
def deckle(bucket, max_combo, max_patterns, target, edge, goal, verbiose):
    patterns, layout = seed_patterns(bucket['w'], 
                                     [math.ceil(i*target) 
                                      for i in bucket['q']], 
                                     bucket['B']+(edge*2), 
                                     bucket['n'], 
                                     max_combinations=max_combo, 
                                     goal=goal, 
                                     verbiose=verbiose)
    loss, inv, summary = find_optimum(patterns, 
                                      layout, 
                                      bucket['w'], 
                                      [math.ceil(i*target) 
                                       for i in bucket['q']], 
                                      bucket['B']+(edge*2), 
                                      bucket['n'], 
                                      bucket['L'],
                                    max_combinations=max_combo,
                                    max_patterns=max_patterns,
                                    prioritize='time')
    return [loss, int(summary['jumbos'].sum()), 
            inv, summary, max_combo, 
            summary.shape[0], target, edge]

In [3]:
buckets = dict()
buckets['A'] = dict()
buckets['A']['w'] = [i+j for i, j in zip([205, 195, 220, 160],
                                         [3, 2, 3, 1])]
buckets['A']['n'] = [3, 2, 3, 1]
buckets['A']['L'] = 17000
buckets['A']['q'] = [776, 557, 470, 17]
buckets['A']['B'] = 4160

buckets['B'] = dict()
buckets['B']['w'] = [i+j for i, j in zip([575, 626, 438, 622, 749, 546],
                                         [10, 11, 9, 11, 14, 10])]
buckets['B']['n'] = [10, 11, 9, 11, 14, 10]
buckets['B']['L'] = 15000
buckets['B']['q'] = [282, 282, 5, 48, 142, 241]
buckets['B']['B'] = 3050

buckets['C'] = dict()
buckets['C']['w'] = [i+j for i, j in zip([622, 749, 800],
                                         [11, 14, 14])]
buckets['C']['n'] = [11, 14, 14]
buckets['C']['L'] = 15000
buckets['C']['q'] = [175, 495, 330]
buckets['C']['B'] = 3050

buckets['D'] = dict()
buckets['D']['w'] = [i+j for i, j in zip([616, 743],
                                         [11, 14])]
buckets['D']['n'] = [11, 14]
buckets['D']['L'] = 15000
buckets['D']['q'] = [850, 150]
buckets['D']['B'] = 3050

buckets['E'] = dict()
buckets['E']['w'] = [1942, 2044, 2146, 2298]
buckets['E']['n'] = [12, 12, 12, 12]
buckets['E']['L'] = 17650
buckets['E']['q'] = [57, 57, 57, 57]
buckets['E']['B'] = 4010


In [31]:
combination_params = [2,3]
pattern_params = [3,4]
production_targets = np.arange(1,2)
edge_trim_allowance = np.arange(1)
goal = 3
verbiose = False
bucket = buckets['A']
res = []
params = []
for bucket in buckets:
    for max_combo in combination_params:
        for max_patterns in pattern_params:
            for target in production_targets:
                for edge in edge_trim_allowance:
                    params.append([buckets[bucket], max_combo, max_patterns, target, edge, goal, verbiose])

In [32]:
# %%timeit
res = []
for param in params:
    res.append(deckle(param[0],
            param[1],
            param[2],
            param[3],
            param[4],
            param[5],
            param[6]))

680 possible max 3 patterns

total loss:      0.46 % (3.02e+04 sqm)
total inventory: 1.64 % (1.08e+05 sqm)

inventory created: {205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}

layout summary:

loss: 1.42% 	 22.0 x	 {205: 0, 195: 20, 220: 0, 160: 1}
loss: 0.26% 	 41.0 x	 {205: 19, 195: 1, 220: 0, 160: 0}
loss: 0.02% 	 30.0 x	 {205: 0, 195: 3, 220: 16, 160: 0}

total jumbos: 93.0 (6.58e+06 sqm)
total loss:      0.13 % (8.74e+03 sqm)
total inventory: 1.96 % (1.29e+05 sqm)

inventory created: {205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}

layout summary:

loss: 0.55% 	 20.0 x	 {205: 0, 195: 21, 220: 0, 160: 0}
loss: 0.14% 	 4.0 x	 {205: 0, 195: 17, 220: 0, 160: 5}
loss: 0.02% 	 30.0 x	 {205: 0, 195: 3, 220: 16, 160: 0}
loss: 0.00% 	 39.0 x	 {205: 20, 195: 0, 220: 0, 160: 0}

total jumbos: 93.0 (6.58e+06 sqm)
680 possible max 3 patterns

total loss:      0.46 % (3.02e+04 sqm)
total inventory: 1.64 % (1.08e+05 sqm)

inventory created: {205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}

layout summary:

loss

In [33]:
df = pd.DataFrame(res)
df.columns = ['loss', 'jumbos', 'inv', 'summary', 'combos', 'layouts', 'target', 'edge']

In [34]:
df

,loss,jumbos,inv,summary,combos,layouts,target,edge
0,0.459833,93,"{205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}",loss jumbos ...,2,3,1,0
1,0.132858,93,"{205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}",loss jumbos ...,2,4,1,0
2,0.459833,93,"{205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}",loss jumbos ...,3,3,1,0
3,0.132858,93,"{205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}",loss jumbos ...,3,4,1,0
4,7.383158,263,"{575: 2.0, 626: 2.0, 438: 66.0, 622: 23.0, 749...",loss jumbos ...,2,3,1,0
5,2.685417,211,"{575: 0.0, 626: 2.0, 438: 5.0, 622: 0.0, 749: ...",loss jumbos ...,2,6,1,0
6,3.923206,225,"{575: 2.0, 626: 2.0, 438: 30.0, 622: 0.0, 749:...",loss jumbos ...,3,3,1,0
7,1.759354,209,"{575: 3.0, 626: 2.0, 438: 1.0, 622: 0.0, 749: ...",loss jumbos ...,3,6,1,0
8,22.122103,319,"{622: 1.0, 749: 0.0, 800: 0.0}",loss jumbos layout 0 ...,2,3,1,0
9,18.383338,305,"{622: 1.0, 749: 2.0, 800: 0.0}",loss jumbos layout 0 ...,2,3,1,0


In [6]:
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply_async(deckle, args=(param[0],
                                    param[1],
                                    param[2],
                                    param[3],
                                    param[4],
                                    param[5],
                                    param[6])) for param in params]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

[<multiprocessing.pool.ApplyResult object at 0x7f3981e28580>, <multiprocessing.pool.ApplyResult object at 0x7f398156c190>, <multiprocessing.pool.ApplyResult object at 0x7f398156c3d0>, <multiprocessing.pool.ApplyResult object at 0x7f398156c4f0>, <multiprocessing.pool.ApplyResult object at 0x7f398156c610>, <multiprocessing.pool.ApplyResult object at 0x7f398156c730>, <multiprocessing.pool.ApplyResult object at 0x7f398156c850>, <multiprocessing.pool.ApplyResult object at 0x7f398156c970>, <multiprocessing.pool.ApplyResult object at 0x7f398156ca90>, <multiprocessing.pool.ApplyResult object at 0x7f398156cbb0>]
816 possible max 3 patterns

816 possible max 3 patterns

816 possible max 3 patterns

816 possible max 3 patterns

816 possible max 3 patterns

816 possible max 3 patterns

total loss:      0.13 % (8.33e+03 sqm)
816 possible max 3 patterns

total inventory: 1.66 % (1.06e+05 sqm)

inventory created: {205: 7.0, 195: 13.0, 220: 8.0, 160: 3.0}

layout summary:

loss: 0.55% 	 19.0 x	 {205: 

loss: 0.29% 	 47.0 x	 {205: 0, 195: 12, 220: 8, 160: 0}
total inventory: 0.60 % (4.05e+04 sqm)

loss: 0.24% 	 4.0 x	 {205: 0, 195: 0, 220: 15, 160: 5}
inventory created: {205: 6.0, 195: 1.0, 220: 0.0, 160: 6.0}


layout summary:

total jumbos: 95.0 (6.72e+06 sqm)
loss: 5.00% 	 42.0 x	 {205: 19, 195: 0, 220: 0, 160: 0}
total loss:      2.30 % (1.58e+05 sqm)
loss: 1.42% 	 24.0 x	 {205: 0, 195: 20, 220: 0, 160: 1}
total inventory: 0.90 % (6.19e+04 sqm)

loss: 0.02% 	 30.0 x	 {205: 0, 195: 3, 220: 16, 160: 0}
inventory created: {205: 10.0, 195: 2.0, 220: 4.0, 160: 2.0}


layout summary:

total jumbos: 96.0 (6.79e+06 sqm)
loss: 4.64% 	 45.0 x	 {205: 18, 195: 0, 220: 1, 160: 0}
total loss:      2.30 % (1.51e+05 sqm)
loss: 0.29% 	 48.0 x	 {205: 0, 195: 12, 220: 8, 160: 0}
total inventory: 0.67 % (4.43e+04 sqm)

loss: 0.24% 	 4.0 x	 {205: 0, 195: 0, 220: 15, 160: 5}
inventory created: {205: 5.0, 195: 0.0, 220: 5.0, 160: 3.0}


layout summary:

total jumbos: 97.0 (6.86e+06 sqm)
loss: 4.64% 	 43

In [7]:
while True:
    if all([i.ready() for i in results]):
        res = []
        for i in results:
            res.append(i.get())
        
        df = pd.DataFrame(res, columns=['loss', 'jumbos', 'inventory', 'summary', 
                                'combinations', 'patterns', 'target', 'edge'])
        # df = df.loc[df['patterns'] < 6]
        df['str summ'] = df['summary'].astype(str)
        df = df.sort_values('loss').reset_index(drop=True)
        df = df.drop_duplicates(['str summ'])\
            [['loss', 'jumbos', 'inventory', 'summary',
             'combinations', 'patterns', 'target', 'edge']].reset_index(drop=True)
        df['loss rank'] = df['loss'].rank(pct=True)
        df['jumbo rank'] = df['jumbos'].rank(pct=True)
        
        break
print(df)

        loss  jumbos                                   inventory  \
0   0.128292      92  {205: 11.0, 195: 5.0, 220: 14.0, 160: 3.0}   
1   0.129438      91   {205: 19.0, 195: 8.0, 220: 3.0, 160: 3.0}   
2   0.130314      95  {205: 8.0, 195: 12.0, 220: 16.0, 160: 2.0}   
3   0.130876      90   {205: 7.0, 195: 13.0, 220: 8.0, 160: 3.0}   
4   0.131444      94  {205: 16.0, 195: 15.0, 220: 5.0, 160: 2.0}   
5   0.132858      93  {205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}   
6   0.134716      96  {205: 0.0, 195: 28.0, 220: 11.0, 160: 2.0}   
7   0.454089      91   {205: 18.0, 195: 2.0, 220: 3.0, 160: 4.0}   
8   0.456197      90    {205: 7.0, 195: 6.0, 220: 8.0, 160: 4.0}   
9   0.459833      93  {205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}   
10  2.298129      93    {205: 5.0, 195: 0.0, 220: 5.0, 160: 3.0}   
11  2.301619      95    {205: 8.0, 195: 1.0, 220: 5.0, 160: 2.0}   
12  2.304966      97   {205: 10.0, 195: 2.0, 220: 4.0, 160: 2.0}   
13  2.549579      96    {205: 6.0, 195: 1.0, 220

In [8]:
df

,loss,jumbos,inventory,summary,combinations,patterns,target,edge,loss rank,jumbo rank
0,0.128292,92,"{205: 11.0, 195: 5.0, 220: 14.0, 160: 3.0}",loss jumbos ...,3,4,0.99,0,0.071429,0.357143
1,0.129438,91,"{205: 19.0, 195: 8.0, 220: 3.0, 160: 3.0}",loss jumbos ...,3,4,0.98,0,0.142857,0.250000
2,0.130314,95,"{205: 8.0, 195: 12.0, 220: 16.0, 160: 2.0}",loss jumbos ...,3,4,1.02,0,0.214286,0.750000
3,0.130876,90,"{205: 7.0, 195: 13.0, 220: 8.0, 160: 3.0}",loss jumbos ...,3,4,0.97,0,0.285714,0.107143
4,0.131444,94,"{205: 16.0, 195: 15.0, 220: 5.0, 160: 2.0}",loss jumbos ...,3,4,1.01,0,0.357143,0.642857
5,0.132858,93,"{205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}",loss jumbos ...,3,4,1.00,0,0.428571,0.500000
6,0.134716,96,"{205: 0.0, 195: 28.0, 220: 11.0, 160: 2.0}",loss jumbos ...,2,4,1.03,0,0.500000,0.892857
7,0.454089,91,"{205: 18.0, 195: 2.0, 220: 3.0, 160: 4.0}",loss jumbos ...,3,3,0.98,0,0.571429,0.250000
8,0.456197,90,"{205: 7.0, 195: 6.0, 220: 8.0, 160: 4.0}",loss jumbos ...,3,3,0.97,0,0.642857,0.107143
9,0.459833,93,"{205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}",loss jumbos ...,3,3,1.00,0,0.714286,0.500000


In [17]:
res = []
for i in results:
    res.append(i.get())

In [19]:
df = pd.DataFrame(res, columns=['loss', 'jumbos', 'inventory', 'summary', 
                                'combinations', 'patterns', 'target', 'edge'])
# df = df.loc[df['patterns'] < 6]
df['str summ'] = df['summary'].astype(str)
df = df.sort_values('loss').reset_index(drop=True)
df = df.drop_duplicates(['str summ'])\
    [['loss', 'jumbos', 'inventory', 'summary',
     'combinations', 'patterns', 'target', 'edge']].reset_index(drop=True)
df['loss rank'] = df['loss'].rank(pct=True)
df['jumbo rank'] = df['jumbos'].rank(pct=True)
df.shape

(14, 10)

In [20]:
df

,loss,jumbos,inventory,summary,combinations,patterns,target,edge,loss rank,jumbo rank
0,0.128292,92,"{205: 11.0, 195: 5.0, 220: 14.0, 160: 3.0}",loss jumbos ...,3,4,0.99,0,0.071429,0.357143
1,0.129438,91,"{205: 19.0, 195: 8.0, 220: 3.0, 160: 3.0}",loss jumbos ...,3,4,0.98,0,0.142857,0.250000
2,0.130314,95,"{205: 8.0, 195: 12.0, 220: 16.0, 160: 2.0}",loss jumbos ...,3,4,1.02,0,0.214286,0.750000
3,0.130876,90,"{205: 7.0, 195: 13.0, 220: 8.0, 160: 3.0}",loss jumbos ...,3,4,0.97,0,0.285714,0.107143
4,0.131444,94,"{205: 16.0, 195: 15.0, 220: 5.0, 160: 2.0}",loss jumbos ...,3,4,1.01,0,0.357143,0.642857
5,0.132858,93,"{205: 4.0, 195: 21.0, 220: 10.0, 160: 3.0}",loss jumbos ...,3,4,1.00,0,0.428571,0.500000
6,0.134716,96,"{205: 0.0, 195: 28.0, 220: 11.0, 160: 2.0}",loss jumbos ...,2,4,1.03,0,0.500000,0.892857
7,0.454089,91,"{205: 18.0, 195: 2.0, 220: 3.0, 160: 4.0}",loss jumbos ...,3,3,0.98,0,0.571429,0.250000
8,0.456197,90,"{205: 7.0, 195: 6.0, 220: 8.0, 160: 4.0}",loss jumbos ...,3,3,0.97,0,0.642857,0.107143
9,0.459833,93,"{205: 3.0, 195: 14.0, 220: 10.0, 160: 5.0}",loss jumbos ...,3,3,1.00,0,0.714286,0.500000


In [11]:
results

[[15.209606692302502,
  243,
  {616: 1.0, 743: 0.0},
        loss jumbos            layout
  0  17.7705    170  {616: 4, 743: 0}
  1   9.2459     73  {616: 2, 743: 2},
  2,
  2,
  0.97,
  0],
 [15.21311475409836,
  245,
  {616: 0.0, 743: 0.0},
        loss jumbos            layout
  0  17.7705    196  {616: 4, 743: 0}
  1  4.98361     49  {616: 1, 743: 3},
  2,
  2,
  0.98,
  0],
 [15.209677419354838,
  248,
  {616: 1.0, 743: 0.0},
        loss jumbos            layout
  0  17.7705     99  {616: 4, 743: 0}
  1  13.5082    149  {616: 3, 743: 1},
  2,
  2,
  0.99,
  0],
 [15.21311475409836,
  250,
  {616: 0.0, 743: 0.0},
        loss jumbos            layout
  0  17.7705    200  {616: 4, 743: 0}
  1  4.98361     50  {616: 1, 743: 3},
  2,
  2,
  1.0,
  0],
 [15.20974535087151,
  253,
  {616: 1.0, 743: 0.0},
        loss jumbos            layout
  0  17.7705    177  {616: 4, 743: 0}
  1   9.2459     76  {616: 2, 743: 2},
  2,
  2,
  1.01,
  0],
 [15.21311475409836,
  255,
  {616: 0.0, 743